In [ ]:
import pathlib
import pandas as pd
import sys
import csv
import time
import emoji
from glob import glob
from os.path import expanduser
from sqlite3 import connect
import os.path
import instaloader
import pandas as pd
from matplotlib import pyplot as plt
from instaloader import ConnectionException, Instaloader
from textblob import TextBlob
import matplotlib.pyplot as plt
import pandas as pd
import instaloader

In [ ]:
!pip install instaloader

In [ ]:
ig=instaloader.Instaloader()

username=input("Enter username:")

#Fetching the details of provided useraname using instaloder object
profile=instaloader.Profile.from_username(ig.context, username)

print("Username: ", profile.username)
print("Number of Posts Uploaded: ", profile.mediacount)
print(profile.username+" is having " + str(profile.followers)+' followers.')
print(profile.username+" is following " + str(profile.followees)+' people')
print("Bio: ", profile.biography)
instaloader.Instaloader().download_profile(username,profile_pic_only=True)

In [ ]:
profile.is_private

In [ ]:
posts = profile.get_posts()
# Iterate and download
for index, post in enumerate(posts, 1):
    ig.download_post(post, target=f"{profile.username}_{index}")

In [ ]:
posts = profile.get_posts()
for post in posts:
  print(post)

In [ ]:
il = instaloader.Instaloader()

try:
    profile = instaloader.Profile.from_username(il.context, username)
    print(f"User {username} is logged in.")
except instaloader.exceptions.LoginRequiredException:
    print("User is not logged in.")

In [ ]:
path_to_firefox_cookies = "C:\\Users\\N Vishnu\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\xw7d8qyg.default-release\\cookies.sqlite"
FIREFOXCOOKIEFILE = glob(expanduser(path_to_firefox_cookies))[0]

In [ ]:
instaloader = Instaloader(max_connection_attempts=1)

In [ ]:
instaloader.context._session.cookies.update(connect(FIREFOXCOOKIEFILE)
                                            .execute("SELECT name, value FROM moz_cookies "
                                                     "WHERE host='.instagram.com'"))

In [ ]:
try:
    username = instaloader.test_login()
    if not username:
        raise ConnectionException()
except ConnectionException:
    raise SystemExit("Cookie import failed. Are you logged in successfully in Firefox?")

In [ ]:
instaloader.context.username = username

In [ ]:
instaloader.save_session_to_file()

In [ ]:
import instaloader

In [ ]:
instagram = instaloader.Instaloader(
    download_pictures=False,
    download_videos=False,
    download_video_thumbnails=False,
    save_metadata=False,
    max_connection_attempts=0
)


In [ ]:
instagram.context.login("nlpuser42", "nlpuser@4655")
instagram.save_session_to_file()

In [ ]:
SHORTCODE = input()

In [ ]:
def scrape_data(url):
    
    post = instaloader.Post.from_shortcode(instagram.context, SHORTCODE)

    output_path = pathlib.Path('Instagram post data')
    output_path.mkdir(parents=True, exist_ok=True)
    
    csvName = SHORTCODE + '.csv'
    post_file = output_path.joinpath(csvName ).open("w", encoding="utf-8")

    field_names = [
                "post_shortcode",
                "commenter_username",
                "comment_text",
                "comment_likes"
                ]

    post_writer = csv.DictWriter(post_file, fieldnames=field_names)
    post_writer.writeheader()

    ## extracting comments from post
    for x in post.get_comments():
        post_info = {
        "post_shortcode":post.shortcode,
        "commenter_username": x.owner,
        "comment_text": (emoji.demojize(x.text)).encode('utf-8', errors='ignore').decode() if x.text else "",
        "comment_likes": x.likes_count
        }
        post_writer.writerow(post_info)

    print("Done Scraping the data and formed Excel sheet!")

In [ ]:
url = input("Enter the URL of the post : ")
scrape_data(url)

In [ ]:
df = pd.read_csv('Instagram post data\\C1qiO9zsPrs.csv')

In [ ]:
def getSentiment(text):
    # Checking if text is not NaN
    if isinstance(text, str):
        return TextBlob(text).sentiment.polarity
    else:
        return None  

# Creating column
df['text_polarity'] = df['comment_text'].apply(getSentiment)

# Convert the 'text_polarity' column to numeric, replacing NaN values with 0
df['text_polarity'] = pd.to_numeric(df['text_polarity'], errors='coerce').fillna(0)

# Defining bins and labels for sentiment categories
bins = [-1, -0.0000000001, 0.0000000001, 1]
labels = ["Negative", "Neutral", "Positive"]

df['sentiment'] = pd.cut(df['text_polarity'], bins=bins, labels=labels)

In [ ]:
df.head()

In [ ]:
graph1 = df.groupby(['post_shortcode', 'sentiment']).count().reset_index()
graph2 = graph1[graph1['post_shortcode'] == SHORTCODE]

colors = ["red", "orange", "green"]

plt.figure(figsize=(18, 9))
plt.bar(graph2["sentiment"], graph2["comment_text"], color=colors)
plt.title("Instagram Sentiment Based on Sentiment", fontsize=20)
plt.xlabel("Sentiment", fontsize=18)
plt.ylabel("Count", fontsize=18)
plt.xticks(rotation=0, fontsize=14)
#plt.yticks(rotation=45, fontsize=14)
plt.show()